# Load Dataset

In [11]:
import pymongo

In [12]:
client = pymongo.MongoClient('localhost',27017)

In [13]:
db = client['chapter3']

In [14]:
collection = db['income']

In [15]:
with open('income_header.txt') as f_in:
    columns_headings = f_in.readline()

In [16]:
columns_headings

'age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,label'

In [17]:
columns_headings_list = columns_headings.split(',')

In [18]:
columns_headings_list

['age',
 'workclass',
 'fnlwgt',
 'education',
 'education-num',
 'marital-status',
 'occupation',
 'relationship',
 'race',
 'sex',
 'capital-gain',
 'capital-loss',
 'hours-per-week',
 'native-country',
 'label']

In [19]:
row_dict_list = list()
with open('income.txt') as f_in:
    for line in f_in:
        row_list = line.rstrip('\n').split(',')
        row_dict = dict(zip(columns_headings_list, row_list))
        row_dict_list.append(row_dict)
        collection.insert_one(row_dict)

In [20]:
collection.count()

32562

# Cleaning

In [21]:
age39 = collection.find_one({'age':39})

In [22]:
age39

In [23]:
row_dict_list[0]

{'age': '39',
 'workclass': ' State-gov',
 'fnlwgt': ' 77516',
 'education': ' Bachelors',
 'education-num': ' 13',
 'marital-status': ' Never-married',
 'occupation': ' Adm-clerical',
 'relationship': ' Not-in-family',
 'race': ' White',
 'sex': ' Male',
 'capital-gain': ' 2174',
 'capital-loss': ' 0',
 'hours-per-week': ' 40',
 'native-country': ' United-States',
 'label': ' <=50K',
 '_id': ObjectId('5feeb9182b498803e6daf2bc')}

In [24]:
collection.find_one({})

{'_id': ObjectId('5feeb9182b498803e6daf2bc'),
 'age': '39',
 'workclass': ' State-gov',
 'fnlwgt': ' 77516',
 'education': ' Bachelors',
 'education-num': ' 13',
 'marital-status': ' Never-married',
 'occupation': ' Adm-clerical',
 'relationship': ' Not-in-family',
 'race': ' White',
 'sex': ' Male',
 'capital-gain': ' 2174',
 'capital-loss': ' 0',
 'hours-per-week': ' 40',
 'native-country': ' United-States',
 'label': ' <=50K'}

In [25]:
collection.delete_many({})
with open('income.txt') as f_in:
    for line in f_in:
        row_list = line.rstrip('\n').split(',')
        row_dict = dict(zip(columns_headings_list, row_list))
        try:
            row_dict['age'] = int(row_dict['age'])
            collection.insert_one(row_dict)
        except:
            pass

In [26]:
over_35 = collection.find({'age': {"$gt" : 35}})

In [27]:
type(over_35)

pymongo.cursor.Cursor

In [28]:
over_35.next()

{'_id': ObjectId('5feeb9eb2b498803e6db71ee'),
 'age': 39,
 'workclass': ' State-gov',
 'fnlwgt': ' 77516',
 'education': ' Bachelors',
 'education-num': ' 13',
 'marital-status': ' Never-married',
 'occupation': ' Adm-clerical',
 'relationship': ' Not-in-family',
 'race': ' White',
 'sex': ' Male',
 'capital-gain': ' 2174',
 'capital-loss': ' 0',
 'hours-per-week': ' 40',
 'native-country': ' United-States',
 'label': ' <=50K'}

In [29]:
over_35.count()

17636

# Creating Indexes

In [30]:
import time

In [34]:
start = time.time()
age50 = collection.find({'age': {'$eq': 50}})
end = time.time()
print(end-start)

0.0


In [35]:
index_result = db.profiles.create_index([('age', pymongo.ASCENDING)], unique=False)

In [36]:
start = time.time()
age45 = collection.find({'age': {'$eq': 45}})
end = time.time()
print(end-start)

0.0


# Creating DataFrames

In [41]:
import pandas as pd

In [42]:
income_df = pd.DataFrame(list(collection.find()))

In [43]:
income_df.head()

,_id,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,label
0,5feeb9eb2b498803e6db71ee,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,5feeb9eb2b498803e6db71ef,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,5feeb9eb2b498803e6db71f0,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,5feeb9eb2b498803e6db71f1,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,5feeb9eb2b498803e6db71f2,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [44]:
income_df.tail()

,_id,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,label
32556,5feeb9ff2b498803e6dbf11a,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
32557,5feeb9ff2b498803e6dbf11b,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
32558,5feeb9ff2b498803e6dbf11c,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
32559,5feeb9ff2b498803e6dbf11d,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K
32560,5feeb9ff2b498803e6dbf11e,52,Self-emp-inc,287927,HS-grad,9,Married-civ-spouse,Exec-managerial,Wife,White,Female,15024,0,40,United-States,>50K


In [46]:
income_df['age'].describe()

count    32561.000000
mean        38.581647
std         13.640433
min         17.000000
25%         28.000000
50%         37.000000
75%         48.000000
max         90.000000
Name: age, dtype: float64

In [47]:
income_df['education'].describe()

count        32561
unique          16
top        HS-grad
freq         10501
Name: education, dtype: object

In [48]:
income_df.mean(0)

age    38.581647
dtype: float64